## Uninformed Search

## Non-deterministic vacuum cleaner


In [10]:
# Dimensions of the plane.
width = 2
height = 1

# The empty cells are marked with 1 meaning dirty cell.
env = [[1 for x in range(width)] for y in range(height)]

# Start point.
start = (0, 0)

# Current state of the environment.
print(f'Current state {env}.')

Current state [[1, 1]].


In [11]:
# Possible moves.
moves = ['Left', 'Right', 'Clean']

# Deterministic effects:
effectD = {}
effectD['Left'] = [(-1, 0, -1, -1, -1, -1)]
effectD['Right'] = [(+1, 0, -1, -1, -1, -1)]
effectD['Clean'] = [(0, 0, 0, -1, 0, -1)]

# Non-deterministic effects:
effectN = {}
effectN['Left'] = effectD['Left']
effectN['Right'] = effectD['Right']
effectN['Clean'] = [(0, 0, 0, -1, 0, -1), (0, 0, 1, -1, 0, 0)]


In [12]:
import operator
from functools import reduce

In [13]:
reduce((lambda x, y: x-y), [-2, -1, 0, 1, 2])

-4

In [14]:
def is_good(state):
    return state[0] >= 0 and state[0] < width and state[1] >= 0 and state[1] < height

def env_clean(env):    
    return 0 == len(list(filter(lambda x: x > 0, reduce(operator.add, env, []))))

def compute_effectD(state, env, move):
    return compute_effects(state, env, move, effectD)

def compute_effectN(state, env, move):
    return compute_effects(state, env, move, effectN)
    
def compute_effects(state, env, move, effects):
    effects = [compute_effect(state, env, effect) for effect in effects[move]]
    effects = list(filter(lambda e: e is not None, effects))
    if len(effects) == 2 and effects[0] == effects[1]:
        return effects[:1]
    return effects
    
def compute_effect(state, env, effect):
    new_env = [line[:] for line in env]
    (x, y) = state
    new_state = tuple([state[idx] + effect[idx] for idx in range(2)])
    if not is_good(new_state):
        return None
    
    for d in range(2):
        clean_effect = effect[2 + d + env[y][x] * 2]
        if clean_effect >= 0 and is_good((x + d, y)):
            new_env[y][x + d] = clean_effect
    return (new_state, new_env)

In [15]:
print(f'Is the environment clean?: {env_clean(env)}')
print(f'What is our move set?: {moves}')
print(f'How does the environment look?: {env}')
print('The vacuum is in the left cell and we try to execute each move in the order stated above:')
print(f'{[compute_effectD((0, 0), env, m) for m in  moves]} -> note that Left is an illegal move here')
print('The vacuum is in the right cell and we try to execute each move in the order stated above:')
print(f'{[compute_effectD((1, 0), env, m) for m in  moves]} -> note that Right is an illegal move here')
print('The vacuum is in the left cell and we try to clean in a non-deterministic environment:')
print(compute_effectN((0, 0), env, 'Clean'))

Is the environment clean?: False
What is our move set?: ['Left', 'Right', 'Clean']
How does the environment look?: [[1, 1]]
The vacuum is in the left cell and we try to execute each move in the order stated above:
[[], [((1, 0), [[1, 1]])], [((0, 0), [[0, 1]])]] -> note that Left is an illegal move here
The vacuum is in the right cell and we try to execute each move in the order stated above:
[[((0, 0), [[1, 1]])], [], [((1, 0), [[1, 0]])]] -> note that Right is an illegal move here
The vacuum is in the left cell and we try to clean in a non-deterministic environment:
[((0, 0), [[0, 1]]), ((0, 0), [[0, 0]])]


### AND/OR Tree Search

In [16]:
class Node:
    def __init__(self, op, state, env, tag=None, children=None):
        # AND / OR type of the node
        self.op = op
        self.state = state
        self.env = env
        self.children = {} if children is None else children
        self.parent = None
        self.tag = tag
    
    def add_child(self, move, child):
        if child == self:
            return
        self.children[move] = child
        child.parent = self
      
    def __str__(self):
        tag = ''
        if self.tag is not None:
            tag = str(self.tag)
        return str(self.op) + " : " + str(self.state) + " : " + str(self.env) + " (" + str(len(self.children.values())) + ") [" + tag + "]"

In [17]:
counter = 0
labels = {}
nodes = []
edges = []

def print_tree(root, onlyOR = False):
    print_tree_ex(root, 0, onlyOR)

def print_tree_ex(node, indent, onlyOR, edge = None):
    global counter
    line = ""
    edge = str(edge) if edge is not None else ""
    for i in range(indent):
        line += "   "
    if node.op == "OR":
        line += "-> "+edge+" <OR> "
        line += str(node.state[0]) + " : " + str(node.env)
    else:
        line += "-> "+edge+" <" + node.op + "> - "
        if onlyOR:
            line += str(node.state[0]) + " : " + str(node.env)
    if node.tag is not None:
        line += " " + str(node.tag)
    print(line)
    counter += 1
    nodes.append(counter)
    if node.parent is not None:
        edges.append((node.parent, counter))
    labels[counter] = line
    for move, child in node.children.items():
        print_tree_ex(child, indent + 1, onlyOR, move)

In [18]:
from IPython.core.display import Image, display, HTML

### Task 0

In [19]:
root = Node("OR",start,env,"Example", children=None)
print(root)

OR : (0, 0) : [[1, 1]] (0) [Example]


In [20]:
def compute_effects_on_node(node, move, deterministic):
    if deterministic:
        effects = compute_effectD(node.state, node.env, move)
    else:
        effects = compute_effectN(node.state, node.env, move)
    return effects

def create_nodes_from_move(parent, move, deterministic=True, give_tag='Example'):
    effects = compute_effects_on_node(parent, move, deterministic)
    children = []
    if len(effects) == 1:
        effect = effects[0]
        child = Node('OR', effect[0], effect[1], give_tag)
        parent.add_child(move, child)
        children = [child]

    if len(effects) > 1:
        and_node = Node('AND', parent.state, parent.env, give_tag)
        parent.add_child(move, and_node)
        for i,effect in enumerate(effects):
            child = Node('OR', effect[0], effect[1], give_tag)
            and_node.add_child(i, child)
            children.append(child)
    return children

In [21]:
child1 = create_nodes_from_move(root,"Right")
create_nodes_from_move(child1[0],"Left")
create_nodes_from_move(child1[0],"Clean")
child2 = create_nodes_from_move(root,"Clean", False)
print_tree(root, False)

->  <OR> 0 : [[1, 1]] Example
   -> Right <OR> 1 : [[1, 1]] Example
      -> Left <OR> 0 : [[1, 1]] Example
      -> Clean <OR> 1 : [[1, 0]] Example
   -> Clean <AND> -  Example
      -> 0 <OR> 0 : [[0, 1]] Example
      -> 1 <OR> 0 : [[0, 0]] Example


In [22]:
viz_states = []
root = Node("OR",start,env,"SOLVED", children=None)
viz_states.append((start,env))
print(viz_states)
print(root.state)    # Aici vad si eu cu ce incep 
print(root.env)
def create_tree(root, viz_states, deterministic):
    
    dicti = {}
    if env_clean(env):
        print('end')
    else:
        for i in moves:
            dicti[i] = len(compute_effects_on_node(root, i, deterministic))
        for j,k in dicti.items():
            if k == 1 and j=='Right':
                child = create_nodes_from_move(root,"Right", True, "SOLVED")
                tuple1 = (child[0].state,child[0].env)
                if tuple1 in viz_states:
                    child = create_nodes_from_move(root,"Right", True, "CYCLE")
                viz_states.append((child[0].state,child[0].env))
                
            elif k == 1 and j == 'Left':
                child = create_nodes_from_move(root,"Left", True, "SOLVED")
                tuple1 = (child[0].state,child[0].env)
                if tuple1 in viz_states:
                    child = create_nodes_from_move(root,"Left", True, "CYCLE")
                viz_states.append((child[0].state,child[0].env))
            elif k == 1 and j == 'Clean':
                child = create_nodes_from_move(root,"Clean", False, "SOLVED")
                tuple1 = (child[0].state,child[0].env)
                if tuple1 in viz_states:
                    child = create_nodes_from_move(root,"Clean", False, "CYCLE")
                viz_states.append((child[0].state,child[0].env))
                if env_clean(child[0].env):
                    print('end')
        print('viz_states:',viz_states)
    print_tree(root, False)


    
create_tree(root,viz_states,True)
print('Until here - the exercise')
print('--------------------------------')
def create_beautiful_tree(root):
    child1 = create_nodes_from_move(root,"Right",True,"SOLVED")
    child99 = create_nodes_from_move(root,"Clean", False,"SOLVED")
    create_nodes_from_move(child1[0],"Left",True,"CYCLE")
    child2 = create_nodes_from_move(child1[0],"Clean",True,"SOLVED")
    child3 = create_nodes_from_move(child2[0],"Left",True,"SOLVED")
    child4 = create_nodes_from_move(child3[0],"Right",True, "CYCLE")
    child5 = create_nodes_from_move(child3[0],"Clean",True, "SOLVED")
    child6 = create_nodes_from_move(child2[0],"Clean", False, "FAIL")
    
    child7 = create_nodes_from_move(child99[0],"Right") #Bored 
    child8 = create_nodes_from_move(child99[0],"Clean",False)
    child9 = create_nodes_from_move(child7[0],"Left")
    child11 = create_nodes_from_move(child7[0],"Clean")
    child10 = create_nodes_from_move(child7[0],"Right")
    print_tree(root,False)
    
create_beautiful_tree(root)









[((0, 0), [[1, 1]])]
(0, 0)
[[1, 1]]
viz_states: [((0, 0), [[1, 1]]), ((1, 0), [[1, 1]]), ((0, 0), [[0, 1]])]
->  <OR> 0 : [[1, 1]] SOLVED
   -> Right <OR> 1 : [[1, 1]] SOLVED
   -> Clean <AND> -  SOLVED
      -> 0 <OR> 0 : [[0, 1]] SOLVED
      -> 1 <OR> 0 : [[0, 0]] SOLVED
Until here - the exercise
--------------------------------
->  <OR> 0 : [[1, 1]] SOLVED
   -> Right <OR> 1 : [[1, 1]] SOLVED
      -> Left <OR> 0 : [[1, 1]] CYCLE
      -> Clean <OR> 1 : [[1, 0]] SOLVED
         -> Left <OR> 0 : [[1, 0]] SOLVED
            -> Right <OR> 1 : [[1, 0]] CYCLE
            -> Clean <OR> 0 : [[0, 0]] SOLVED
         -> Clean <AND> -  FAIL
            -> 0 <OR> 1 : [[1, 0]] FAIL
            -> 1 <OR> 1 : [[1, 1]] FAIL
   -> Clean <AND> -  SOLVED
      -> 0 <OR> 0 : [[0, 1]] SOLVED
         -> Right <OR> 1 : [[0, 1]] Example
            -> Left <OR> 0 : [[0, 1]] Example
            -> Clean <OR> 1 : [[0, 0]] Example
         -> Clean <AND> -  Example
            -> 0 <OR> 0 : [[0, 1]] Examp